In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL de base
BASE_URL = "https://www.expat-dakar.com/vehicules"


In [ ]:
# Headers pour simuler un navigateur
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}


In [ ]:
# Fonction pour récupérer les liens des annonces sur une page donnée
def get_announcement_links(base_url, page):
    response = requests.get(f"{base_url}?page={page}", headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Extraire tous les liens des annonces
    links = []
    for item in soup.find_all("div", class_="listings-cards__list-item"):
        link_tag = item.find("a", href=True)
        if link_tag:
            href = link_tag['href']
            # Vérifiez que le lien est bien formaté
            if href.startswith("/"):  # Lien relatif
                links.append("https://www.expat-dakar.com" + href)
            elif href.startswith("http"):  # Lien absolu déjà formé
                links.append(href)
    
    return links

In [ ]:
# Fonction pour récupérer les détails d'une annonce
def get_announcement_details(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Récupérer les informations spécifiques
    details = {}
    try:
        # Titre et prix
        details["Titre"] = soup.find("h1", class_="listing-item__header").text.strip()
        details["Prix"] = soup.find("span", class_="listing-card__price__value").text.strip()
        
        # Adresse
        location = soup.find("span", class_="listing-item__address-location")
        region = soup.find("span", class_="listing-item__address-region")
        details["Adresse"] = f"{location.text.strip()} {region.text.strip()}" if location and region else "N/A"
        
        # Récupérer les détails listés dans "Détails"
        for dt, dd in zip(
            soup.find_all("dt", class_="listing-item__properties__title"),
            soup.find_all("dd", class_="listing-item__properties__description")
        ):
            details[dt.text.strip()] = dd.text.strip()
    except Exception as e:
        print(f"Erreur lors de l'extraction de {url}: {e}")
    
    return details

# Récupérer tous les liens et les détails des annonces sur plusieurs pages
print("Début du scraping des annonces...")
all_links = []
data = []  # Pour stocker les détails des annonces

# Boucle pour parcourir les pages 
MAX_PAGES = 5  
for page in range(1, MAX_PAGES + 600):
    print(f"Scraping de la page {page}...")
    links = get_announcement_links(BASE_URL, page)
    all_links.extend(links)
    print(f"{len(links)} liens trouvés sur la page {page}.")
    
    # Récupérer les détails pour chaque annonce de cette page
    for link in links:
        print(f"Scraping des détails pour : {link}")
        details = get_announcement_details(link)
        data.append(details)
        time.sleep(1)  # Pause pour éviter de surcharger le serveur

# Sauvegarder les résultats dans un fichier Excel
df = pd.DataFrame(data)
df.to_excel("annonces_expat_dakar_pagination.xlsx", index=False)
print("Les données ont été sauvegardées dans 'annonces_expat_dakar_pagination.xlsx'.")


Début du scraping des annonces...
Scraping de la page 1...
11 liens trouvés sur la page 1.
Scraping des détails pour : https://www.expat-dakar.com/annonce/volkswagen-atlas-premium-2018-6239740
Scraping des détails pour : https://www.expat-dakar.com/annonce/kia-k7-anne-2014-full-options-6241091
Scraping des détails pour : https://www.expat-dakar.com/annonce/mitsubishi-rvr-2014-6241086
Scraping des détails pour : https://www.expat-dakar.com/annonce/hyundai-6241061
Scraping des détails pour : https://www.expat-dakar.com/annonce/location-ford-fusion-6103399
Scraping des détails pour : https://www.expat-dakar.com/annonce/dodge-2012-6123064
Scraping des détails pour : https://www.expat-dakar.com/annonce/toyota-rav4-6241073
Scraping des détails pour : https://www.expat-dakar.com/annonce/renault-sm6-2016-6241072
Scraping des détails pour : https://www.expat-dakar.com/annonce/edge-v6-2015-6190963
Scraping des détails pour : https://www.expat-dakar.com/annonce/glc-300-4matic-6213177
Scraping des